In [5]:
import requests
from bs4 import BeautifulSoup
import re

import pandas as pd

def recuperer_contenu(url, nom_classe):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Erreur lors de la requête : {e}")
        return []
    
    soup = BeautifulSoup(response.text, 'html.parser')
    balises = soup.find_all(class_=nom_classe)
    
    return balises

In [6]:
# Recuperation des url 

import json
def read_json_file(file_path):
    try:
        # Ouverture et lecture du fichier JSON
        with open(file_path, 'r', encoding='utf-8') as file:
            # Conversion du contenu JSON en dictionnaire Python
            data = json.load(file)
            return data
    except FileNotFoundError:
        print(f"Erreur: Le fichier {file_path} n'existe pas")
        return None
    except json.JSONDecodeError:
        print(f"Erreur: Le fichier {file_path} n'est pas un JSON valide")
        return None
    except Exception as e:
        print(f"Une erreur inattendue s'est produite: {str(e)}")
        return None

def get_ship(data, name):
    return data[name]
    

def get_url(data,name):
    url = get_ship(data,name)['pledge_url']
    return url


data = read_json_file('shipDB_All.json')

# creation d une liste contenant toutes les url 
 
def list_ship(data):
    Flight_ready=[]
    In_Concept = []
    for i in data:
        if 'pledge_url' in data[i].keys():
            Flight_ready.append(i)
        else : 
            In_Concept.append(i)
    return Flight_ready,In_Concept 
    

In [7]:
Flight_ready = [get_ship(data,i)['pledge_url']  for i in list_ship(data)[0]] 

In [18]:
from collections import defaultdict

def fusion_complete(*dicts):
    resultat = defaultdict(list)
    for d in dicts:
        for cle, valeur in d.items():
            resultat[cle].append(valeur)
    
    return dict(resultat)

# Exemple avec types complexes
dict1 = {'a': [1, 2], 'b': {'x': 1}, 'c': {1, 2}}
dict2 = {'a': [1, 2], 'b': {'x': 2}, 'c': {2, 3}}
resultat = fusion_complete(dict1, dict2)
resultat

{'a': [[1, 2], [1, 2]], 'b': [{'x': 1}, {'x': 2}], 'c': [{1, 2}, {2, 3}]}

In [19]:
#creation du dataframe

def DataBase():
    nom_classe = "l-sheet__block"
    dict_ = {}
    for i in Flight_ready[:5]:
        dict = {}
        for e in recuperer_contenu(i, nom_classe):
            e = e.text.replace("\n", "|").replace("\t", "")
            e = e.split("|")
            n = []
            for i in e:
                i = re.sub(r'\s+', ' ', i).strip()
                if len(i)!=0:
                    n.append(i)
            for i in range(0, len(n)-2, 2):
                dict[n[i]] = n[i+1]
        print(dict)
        dict_ = fusion_complete(dict_,dict)
        
    return dict_

DB = DataBase()

{'Focus': 'Starter / Touring', 'Length': '19.3m', 'Beam': '11.0m', 'Height': '4.0m', 'Size': 'Small', 'Cargo Capacity': '2', 'SCM Speed': '260 m/s', 'Afterburner Speed': '1,425 m/s', 'Min Crew': '1', 'Pitch Max': '- deg/s', 'Yaw Max': '- deg/s', 'Roll Max': '- deg/s', 'X-Axis Acceleration': '- m/s/s', 'Y-Axis Acceleration': '- m/s/s'}
{'Focus': 'Starter / Light Fighter', 'Length': '19.3m', 'Beam': '11.0m', 'Height': '4.0m', 'Size': 'Small', 'Cargo Capacity': '2', 'SCM Speed': '260 m/s', 'Afterburner Speed': '1,425 m/s', 'Min Crew': '1', 'Pitch Max': '- deg/s', 'Yaw Max': '- deg/s', 'Roll Max': '- deg/s', 'X-Axis Acceleration': '- m/s/s', 'Y-Axis Acceleration': '- m/s/s'}
{'Focus': 'Starter / Light Freight', 'Length': '19.3m', 'Beam': '11.0m', 'Height': '4.0m', 'Size': 'Small', 'Cargo Capacity': '6', 'SCM Speed': '260 m/s', 'Afterburner Speed': '1,425 m/s', 'Min Crew': '1', 'Pitch Max': '- deg/s', 'Yaw Max': '- deg/s', 'Roll Max': '- deg/s', 'X-Axis Acceleration': '- m/s/s', 'Y-Axis Acc

In [20]:
DB # probleme sur les valeurs: liste imbriquées

{'Focus': [[[[['Starter / Touring'], 'Starter / Light Fighter'],
    'Starter / Light Freight'],
   'Luxury / Touring'],
  'Pathfinder'],
 'Length': [[[[['19.3m'], '19.3m'], '19.3m'], '27.0m'], '27.0m'],
 'Beam': [[[[['11.0m'], '11.0m'], '11.0m'], '17.0m'], '17.0m'],
 'Height': [[[[['4.0m'], '4.0m'], '4.0m'], '8.0m'], '8.0m'],
 'Size': [[[[['Small'], 'Small'], 'Small'], 'Small'], 'Small'],
 'Cargo Capacity': [[[[['2'], '2'], '6'], '8'], '12'],
 'SCM Speed': [[[[['260 m/s'], '260 m/s'], '260 m/s'], '260 m/s'], '260 m/s'],
 'Afterburner Speed': [[[[['1,425 m/s'], '1,425 m/s'], '1,425 m/s'],
   '1,425 m/s'],
  '1,425 m/s'],
 'Min Crew': [[[[['1'], '1'], '1'], '1'], '1'],
 'Pitch Max': [[[[['- deg/s'], '- deg/s'], '- deg/s'], '- deg/s'], '- deg/s'],
 'Yaw Max': [[[[['- deg/s'], '- deg/s'], '- deg/s'], '- deg/s'], '- deg/s'],
 'Roll Max': [[[[['- deg/s'], '- deg/s'], '- deg/s'], '- deg/s'], '- deg/s'],
 'X-Axis Acceleration': [[[[['- m/s/s'], '- m/s/s'], '- m/s/s'], '- m/s/s'],
  '- m/s/s']

In [ ]:
DF = pd.DataFrame(DB)
DF

,Focus,Length,Beam,Height,Size,Cargo Capacity,SCM Speed,Afterburner Speed,Min Crew,Pitch Max,Yaw Max,Roll Max,X-Axis Acceleration,Y-Axis Acceleration
0,"[[Starter / Touring], Starter / Light Fighter]","[[19.3m], 19.3m]","[[11.0m], 11.0m]","[[4.0m], 4.0m]","[[Small], Small]","[[2], 2]","[[260 m/s], 260 m/s]","[[1,425 m/s], 1,425 m/s]","[[1], 1]","[[- deg/s], - deg/s]","[[- deg/s], - deg/s]","[[- deg/s], - deg/s]","[[- m/s/s], - m/s/s]","[[- m/s/s], - m/s/s]"
1,Starter / Light Freight,19.3m,11.0m,4.0m,Small,6,260 m/s,"1,425 m/s",1,- deg/s,- deg/s,- deg/s,- m/s/s,- m/s/s


In [10]:
vaisseaux =  recuperer_contenu('https://scfocus.org/ship-sale-rental-locations-history/',nom_classe= 'column-1')
prix= recuperer_contenu('https://scfocus.org/ship-sale-rental-locations-history/',nom_classe= 'column-2')

def url_to_list(D):
    L =[]
    for e in D:
        e = e.text.replace("\n", "|").replace("\t", "")
        e = e.split("|")
        L += e
    return L[1:]

caca = [i for i in zip(url_to_list(vaisseaux),url_to_list(prix))]
pipi = list(set(caca))
pipi

[('Cutter', '635,040'),
 ('Constellation Aquila', '12,186,720'),
 ('Aurora ES', '8,67 (2,482)'),
 ('Cyclone MT', '141,750'),
 ('Cutlass Black', '2,116,800'),
 ('Valkyrie', '19,845,000'),
 ('X1 Velocity', '119,020'),
 ('Talon', '3,260,250'),
 ('Mustang Beta', '655,200'),
 ('Dragonfly Yellowjacket', '342,720'),
 ('125a', '1,587,600'),
 ('MPUV Cargo', '330,750'),
 ('135c', '1,490,580'),
 ('Constellation Phoenix', '247,506'),
 ('Terrapin', '5,433,120'),
 ('Avenger Titan', '27,165'),
 ('C2 Starlifter', '18,900,000'),
 ('Mantis', '3,402,000'),
 ('600i', '27,231,750'),
 ('Nox', '21,687 (10,476)'),
 ('Freelancer DUR', '3,316,950'),
 ('Avenger Titan', '1,358,280'),
 ('Hull A', '50,808'),
 ('Greycat ROC', '103,950'),
 ('Banu Defender', '6,237,000'),
 ('315p', '1,490,580'),
 ('MISC Razor EX', '1,757,700'),
 ('Eclipse', '7,938,000'),
 ('Reliant Tana', '1,984,500'),
 ('Ballista', '1,481,760'),
 ('ALTS', '75,600'),
 ('Herald', '1,285,200'),
 ('Avenger Titan Renegade', '1,852,200'),
 ('Sabre Firebird

In [11]:
recuperer_contenu('https://robertsspaceindustries.com/en/pledge/ships/origin-100/100i',nom_classe='statbox title')

[]